In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import os

def compute_vgg16_descriptors(image_folder):
    model = VGG16(weights='imagenet', include_top=False, pooling='avg')
    descriptors = np.zeros((0,512))

    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(image_folder, filename)
            img = image.load_img(img_path, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            
            features = model.predict(img_data)
            r = features.flatten()
            
            descriptors = np.vstack([descriptors, r])

    return descriptors



In [4]:
descriptors.shape

(600, 512)

In [5]:
np.save('./data/nordland/raw/summer', descriptors)

In [56]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
import numpy as np
import os

def compute_custom_vgg16_descriptors(image_folder):
    base_model = VGG16(weights='imagenet', include_top=False)
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(4096, activation='relu'),  # Adding a dense layer with 4096 units
        Dense(4096, activation='relu')   # Another dense layer to increase dimensionality
    ])
    
    descriptors = np.zeros((0,4096))

    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(image_folder, filename)
            img = image.load_img(img_path, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            
            features = model.predict(img_data)
            r = features.flatten()
            descriptors = np.vstack([descriptors, r])

    return descriptors



# Output is suppressed, and descriptors are stored in the `descriptors` dictionary

In [60]:
np.save('./data/nordland/raw/summer', descriptors)

In [62]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

def compute_high_dimensional_descriptors(image_folder, output_dim=4096):
    # Load the VGG16 model without the top layers
    base_model = VGG16(weights='imagenet', include_top=False)
    
    # Create a custom model with an additional Dense layer with tanh activation
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(output_dim, activation='tanh')  # 4096 dimensions with tanh activation
    ])
    
    descriptors = np.zeros((0,4096))

    for filename in sorted(os.listdir(image_folder)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            img_path = os.path.join(image_folder, filename)
            img = image.load_img(img_path, target_size=(224, 224))
            img_data = image.img_to_array(img)
            img_data = np.expand_dims(img_data, axis=0)
            img_data = preprocess_input(img_data)
            
            # Compute the descriptor
            features = model.predict(img_data)
            r = features.flatten()
            descriptors = np.vstack([descriptors, r])

    return descriptors

## Specify the directory containing images
#image_folder = '../images/nordland/summer'

## Compute the high-dimensional descriptors
#descriptors = compute_high_dimensional_descriptors(image_folder)

#np.save('./data/nordland/raw/summer', descriptors)


In [65]:
descriptors.shape

(600, 4096)

In [66]:
np.save('./data/nordland/raw/summer', descriptors)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pickle

# Load the DELF model from TensorFlow Hub
delf = hub.load('https://tfhub.dev/google/delf/1').signatures['default']

def compute_delf_descriptors(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = tf.image.resize(img_rgb, (224, 224))  # Resize the image
    img_resized = tf.image.convert_image_dtype(img_resized, tf.float32)
    
    # Compute DELF features
    result = delf(image=tf.expand_dims(img_resized, axis=0))
    
    # Extract the descriptors
    descriptors = result['descriptors'].numpy()
    keypoints = result['keypoints'].numpy()
    
    return keypoints, descriptors

def process_images_in_folder(folder_path):
    descriptors_list = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            keypoints, descriptors = compute_delf_descriptors(image_path)
            descriptors_list.append((filename, keypoints, descriptors))
    
    return descriptors_list



In [ ]:
# Specify the path to your images folder
folder_path = '/path/to/your/images'

# Process the images
descriptors_list = process_images_in_folder(folder_path)



In [ ]:
# Save descriptors and keypoints to a file
with open('delf_descriptors.pkl', 'wb') as f:
    pickle.dump(descriptors_list, f)

# Load the descriptors later if needed
with open('delf_descriptors.pkl', 'rb') as f:
    loaded_descriptors = pickle.load(f)

In [6]:
import os
import torch
import cv2
import numpy as np
import torchvision.transforms as transforms
import torchvision.models as models
from torch import nn
import pickle

import torch.nn as nn
import torch.nn.functional as F

class NetVLAD(nn.Module):
    def __init__(self, num_clusters=64, dim=512):
        super(NetVLAD, self).__init__()
        self.num_clusters = num_clusters
        self.dim = dim
        self.conv = nn.Conv2d(dim, num_clusters, kernel_size=(1, 1), bias=True)
        self.centroids = nn.Parameter(torch.rand(num_clusters, dim))
    
    def forward(self, x):
        N, C, H, W = x.shape  # N is batch size, C is the number of channels, H and W are height and width
        soft_assign = self.conv(x)  # [N, num_clusters, H, W]
        soft_assign = soft_assign.view(N, self.num_clusters, -1)  # [N, num_clusters, H*W]
        soft_assign = F.softmax(soft_assign, dim=1)  # Apply softmax along the clusters
        
        x_flatten = x.view(N, C, -1)  # Flatten the feature maps to [N, C, H*W]
        x_flatten = x_flatten.permute(0, 2, 1)  # Change to [N, H*W, C] for easier broadcasting
        
        vlad = torch.zeros([N, self.num_clusters, C], dtype=x.dtype, layout=x.layout, device=x.device)
        for Ck in range(self.num_clusters):
            residual = x_flatten - self.centroids[Ck:Ck + 1, :].expand(x_flatten.size(0), -1, -1)  # [N, H*W, C]
            residual *= soft_assign[:, Ck:Ck + 1, :].permute(0, 2, 1)  # [N, H*W, 1]
            vlad[:, Ck:Ck + 1, :] = residual.sum(dim=1)  # Sum over H*W dimension
            
        vlad = F.normalize(vlad, p=2, dim=2)  # intra-normalization
        vlad = vlad.view(N, -1)  # Flatten to [N, num_clusters*C]
        vlad = F.normalize(vlad, p=2, dim=1)  # L2 normalize
        
        return vlad

def load_netvlad_model():
    backbone = models.vgg16(pretrained=True)
    netvlad_layer = nn.Sequential(
        *list(backbone.features.children())[:-2],
        NetVLAD(num_clusters=64)
    )
    return netvlad_layer

def compute_netvlad_descriptor(image_path, model):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (224, 224))
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    img_tensor = transform(img_rgb).unsqueeze(0)
    
    with torch.no_grad():
        descriptor = model(img_tensor)
    
    return descriptor.cpu().numpy()





In [7]:
des = compute_netvlad_descriptor('../images/nordland/spring/nl_spring00001.png', load_netvlad_model())

In [44]:
import numpy as np
from sklearn.random_projection import GaussianRandomProjection

rng = np.random.RandomState(42)
X = rng.rand(25, 3000)
transformer = GaussianRandomProjection(random_state=rng)
X_new = transformer.fit_transform(X)
X_new.shape

(25, 2759)

In [24]:
des[0].shape

(32768,)

In [25]:
def process_images_in_folder(folder_path, model):
    descriptors_list = []
    
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            descriptor = compute_netvlad_descriptor(image_path, model)
            descriptors_list.append(descriptor[0])
    
    return np.array(descriptors_list)

In [43]:
from sklearn.preprocessing import normalize
def preprocess_descriptors(desc_matr):
    #gaussian random projection
    transformer = GaussianRandomProjection()
    desc_matr_new = transformer.fit_transform(desc_matr)
    #l2 normalization
    desc_matr_new = normalize(desc_matr_new, axis=1, norm='l2')
    #dimensionwise standardization to standard normal distributions
        # Step 1: Compute the mean and standard deviation for each column
    mean = np.mean(desc_matr_new, axis=0)  # Mean along columns
    std_dev = np.std(desc_matr_new, axis=0)  # Standard deviation along columns

        # Step 2: Standardize the columns
    desc_matr_standardized = (desc_matr_new - mean) / std_dev
    return desc_matr_standardized

In [45]:
from time import perf_counter
folder_path = '../images/nordland/spring'
model = load_netvlad_model()
start = perf_counter()
descriptors_list = process_images_in_folder(folder_path, model)
end = perf_counter()
duration = end - start
print(f"Duration creation descriptors: {duration} seconds")




/Users/bz/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/bz/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Duration creation descriptors: 41.70160608399601 seconds


In [46]:
start = perf_counter()
descriptors_list_pp = preprocess_descriptors(descriptors_list)
end = perf_counter()
duration = end - start
print(f"Duration creation descriptors: {duration} seconds")


Duration creation descriptors: 3.373913374991389 seconds


In [47]:
descriptors_list_pp.shape

(600, 5483)

In [48]:
np.save('./data/nordland/raw/spring', descriptors_list_pp)

In [49]:
from time import perf_counter
folder_path = '../images/nordland/winter'
model = load_netvlad_model()
start = perf_counter()
descriptors_list = process_images_in_folder(folder_path, model)
end = perf_counter()
duration = end - start
print(f"Duration creation descriptors: {duration} seconds")
start = perf_counter()
descriptors_list_pp = preprocess_descriptors(descriptors_list)
end = perf_counter()
duration = end - start
print(f"Duration creation descriptors: {duration} seconds")


/Users/bz/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/bz/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Duration creation descriptors: 39.1135114159988 seconds
Duration creation descriptors: 3.3795906659943284 seconds


In [52]:
np.save('./data/nordland/raw/winter', descriptors_list_pp)

In [53]:
descriptors_list_pp.shape

(600, 5483)